In [2]:
import sys
!{sys.executable} -m pip install pandas

You should consider upgrading via the 'C:\Users\pruet\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [4]:
import json
import pprint
import urllib
import pandas as pd
import requests

# read configuration
config_file = 'config.json'
config = json.load(open(config_file))
api_key = config['api_key']
server = config['server']


In [5]:
def send_request_to_server(server, api_key, action_path, dataset_dict):
# Make the HTTP request.
	headers_dict = {
		'Authorization': api_key,
		'Content-Type' : 'application/x-www-form-urlencoded'
	}
	dataset_dict = urllib.parse.quote(json.dumps(dataset_dict))
	req = requests.post(server + 'api/action/' + action_path, headers=headers_dict, data=dataset_dict)

	# Use the json module to load CKAN's response into a dictionary.
	response_dict = req.json()

	# Check the contents of the response.
	#assert response_dict['success'] is True
	#result = response_dict['result']
	pprint.pprint(response_dict)


In [6]:
def create_package(server, api_key, dataset_dict):
	send_request_to_server(server, api_key, 'package_create', dataset_dict)

In [7]:
def update_package(server, api_key, dataset_dict):
	send_request_to_server(server, api_key, 'package_update', dataset_dict)

In [14]:
df = pd.read_csv('ckan_data.csv')
for index, row in df.iterrows():
	tags = row['Tags'].split(',')
	tag_list= []
	for tag in tags:
		tag_list.append({'name' : tag})
	dataset_dict = {
		'owner_org' : row['Organization'],
		'groups' : [
			{'id' : row['Group']}
		],
		'name' : row['Name'],
		'title' : row['Title'],
		'author' : row['Author'],
		'notes' : row['Notes'],
		'author_email' : row['AuthorEmail'],
		'maintainer' : row['Maintainer'],
		'maintainer_email' : row['MaintainerEmail'],
		'url' : row['Url'],
		'tags' : tag_list,
		'extras' : [
			{'key' : 'PublicAccess', 'value' : row['PublicAccess']},
			{'key' : 'UniversityAccess', 'value' : row['UniversityAccess']},
			{'key' : 'FacultyAccess', 'value' : row['FacultyAccess']},
			{'key' : 'DepartmentAccess', 'value' : row['DepartmentAccess']},
			{'key' : 'AdvisorAccess', 'value' : row['AdvisorAccess'] },
			{'key' : 'StudentAccess', 'value' : row['StudentAccess']},
			{'key' : 'PDPA', 'value' : row['PDPA']},
			{'key' : 'RetainPolicy', 'value' : row['ReatainPolicy'] },
			{'key' : 'UpdatePolicy', 'value' : row['UpdatePolicy'] }
		]
	}
	update_package(server, api_key, dataset_dict)

KeyError: 'Groups'